### This code is used to check and test images on a classifier. However, the classifier only classifies correctly 50% of the time. This is because the transformation from numpy arrays to PNG images can affect the quality and colors of the images, resulting in the input images not being exactly the same as the ones used for testing.

In [1]:
import torch
from torchvision.transforms import Compose, ToTensor
from PIL import Image
import sys
from pathlib import Path
import os

In [5]:
def preprocess_image(input_image: Image.Image):
    transforms = Compose([
        ToTensor()
    ])
    img_tensor = transforms(input_image)
    img_tensor = img_tensor.unsqueeze(0)
    return img_tensor

# Constructs a path to a directory that contains dataloader.py and plotCreator.py
module_path = str(Path.cwd().parents[0] / "methods")

# Checks to see if the directory is already in sys.path to avoid adding it multiple times.
if module_path not in sys.path:
    sys.path.append(module_path)
import nnmodel

model = nnmodel.VGGNet2(0.5)
model_path = os.path.join(os.getcwd(), "models", "best_model3.pth")
model.load_state_dict(torch.load(model_path))
model.eval()


def predict(input_image: Image.Image):
    img_tensor = preprocess_image(input_image)
    with torch.no_grad():
        output = model(img_tensor)
        _, prediction = torch.max(output, 1)
    
    class_label = "Black Hole" if prediction.item() == 0 else "Sphaleron"
    return class_label

In [6]:
def predict_all_images_in_folder(folder_path: str):
    file_names = os.listdir(folder_path)
    results = {"True Black Hole": 0, "True Sphaleron": 0, "False Black Hole": 0, "False Sphaleron": 0}

    for file_name in file_names:
        if file_name.endswith(".png"):
            image_path = os.path.join(folder_path, file_name)
            image = Image.open(image_path)
            prediction = predict(image)

            if "bh" in file_name:
                if prediction == "Black Hole":
                    results["True Black Hole"] += 1
                else:
                    results["False Sphaleron"] += 1
            elif "sph" in file_name:
                if prediction == "Sphaleron":
                    results["True Sphaleron"] += 1
                else:
                    results["False Black Hole"] += 1

    return results

In [7]:
current_directory = os.getcwd()
blackhole_folder_path = os.path.join(current_directory, 'output', 'black_holes')
sphaleron_folder_path = os.path.join(current_directory, 'output', 'sphalerons')

black_hole_results = predict_all_images_in_folder(blackhole_folder_path)
sphaleron_results = predict_all_images_in_folder(sphaleron_folder_path)

def calculate_confusion_matrix(blackhole_folder_path, sphaleron_folder_path):
    black_hole_results = predict_all_images_in_folder(blackhole_folder_path)
    sphaleron_results = predict_all_images_in_folder(sphaleron_folder_path)

    confusion_matrix = {
        "True Black Hole": black_hole_results["True Black Hole"],
        "False Black Hole": sphaleron_results["False Black Hole"],
        "True Sphaleron": sphaleron_results["True Sphaleron"],
        "False Sphaleron": black_hole_results["False Sphaleron"]
    }

    return confusion_matrix

confusion_matrix = calculate_confusion_matrix(blackhole_folder_path, sphaleron_folder_path)

print("Confusion Matrix:")
print(confusion_matrix)

Confusion Matrix:
{'True Black Hole': 100, 'False Black Hole': 100, 'True Sphaleron': 0, 'False Sphaleron': 0}
